In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import glob
from datetime import datetime as dt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [30]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

In [11]:
df_sf_2017 = pd.read_json('df_sf_2017')

In [54]:
# df_sf['price'].apply(np.log)[df_sf_2017['price_per_guest'].isnull()][[ 'price', 'guests_included','price_per_guest']]

,price,guests_included,price_per_guest


In [3]:
df_sf_2017.price_per_bedroom.isnull().sum()

0

## Random Forest

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pickle

## Predict 2017

In [12]:
def split_data(start_month, end_month):
    df_sf_temp = df_sf_2017.copy()
    X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
    y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

    X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    
    return X_train, y_train, X_test, y_test
    

In [17]:
def transform_data(X):
    X['extra_people'].replace('[\=$,]', '', regex=True, inplace=True)
#     X['price'].replace('[\=$,]', '', regex=True, inplace=True)
    X['property_type'] = le.fit_transform(X['property_type'])
    X['bed_type'] = le.fit_transform(X['bed_type'])
    X['cancellation_policy'] = le.fit_transform(X['cancellation_policy'])
    X['room_type'] = le.fit_transform(X['room_type'])
 
    return X

In [14]:
def predict_data(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
    scores[0][model_num] = accuracy_score(y_test, predicted)
    scores[1][model_num] = recall_score(y_test, rf.predict(X_test))
    scores[2][model_num] = precision_score(y_test, rf.predict(X_test))
    scores[3][model_num] = f1_score(y_test, rf.predict(X_test))
    return scores

In [18]:
scores = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type']

In [ ]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    le = LabelEncoder()
    X_train = transform_data(X_train)
    X_test = transform_data(X_test)
    scores = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)

1 4 0
[[0.86274284 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.39735516 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.72695853 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51384365 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
[[0.86274284 0.92304169 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.39735516 0.64398236 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.72695853 0.90522586 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51384365 0.75257732 0.         0.         0.         0.
  0.         0.         0.        ]]
3 6 2
[[0.86274284 0.92304169 0.92224622 0.         0.         0.
  0.         0.         0.        ]
 [0.39735516 0.64398236 0.65985577 0.         0.         0.
  0.         0.         0.        ]
 [0.72695853 0.90522

In [51]:
from tempfile import TemporaryFile
scores_rf_baseline = TemporaryFile()
np.save(scores_rf_baseline,scores)x

In [53]:
scores_rf_baseline.seek(0) # Only needed here to simulate closing & reopening file
np.load(scores_rf_baseline)

array([[0.81988506, 0.89212093, 0.88964655, 0.88999322, 0.88094975,
        0.88061452, 0.88234636, 0.88732079, 0.85575529],
       [0.2336272 , 0.47006931, 0.48648649, 0.4973822 , 0.47533875,
        0.49067661, 0.49973698, 0.53308824, 0.5445224 ],
       [0.51456311, 0.88075561, 0.87473002, 0.88601036, 0.88855117,
        0.93692777, 0.9047619 , 0.89664311, 0.86982759],
       [0.32135123, 0.61298274, 0.62524122, 0.63710879, 0.61935028,
        0.64405594, 0.64384954, 0.66864295, 0.66976435]])

In [ ]:
accuracy_2017 = np.mean(scores[0])
recall_2017 = np.mean(scores[1])
precision_2017 = np.mean(scores[2])
f1_score_2017 = np.mean(scores[3])
print(accuracy_2017)
print(recall_2017)
print(precision_2017)
print(f1_score_2017)